In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = spark.sql('select duns, date from workarea.us_expprop_exporters_build_201811')

df = df.withColumn('date_preprocessed',  to_date(unix_timestamp(col('date'), 'MM/dd/yyyy').cast('timestamp')))

df = df.withColumn('append_year', lit(None))
df = df.withColumn('append_month', lit(9))
df = df.withColumn('append_year', when(col('date_preprocessed') < lit("2017-10-01"), 2016).otherwise(col('append_year')))
df = df.withColumn('append_year', when(col('date_preprocessed') >= lit("2017-10-01"), 2017).otherwise(col('append_year')))

df = df.withColumn('export', lit(1))

wblinkage = spark.sql('select duns_nbr as duns, phy_ctry_code as country_code, load_year as append_year, load_month as append_month from dnb_data_globalarea.wblinkage where load_year in (2016, 2017) and load_month = 9')
wbusunlinked = spark.sql('select duns_nbr as duns, phy_ctry_code as country_code, load_year as append_year, load_month as append_month from dnb_data_globalarea.wbusunlinked where load_year in (2016, 2017) and load_month = 9')
wbglobalunlinked = spark.sql('select duns_nbr as duns, phy_ctry_code as country_code, load_year as append_year, load_month as append_month from dnb_data_globalarea.wbglobalunlinked where load_year in (2016, 2017) and load_month = 9')

wb = wblinkage.union(wbusunlinked).union(wbglobalunlinked)

df2 = df.join(wb.dropDuplicates(['duns', 'append_year', 'append_month']), on = ['duns', 'append_year', 'append_month'], how = 'inner')

df3 = df2.where('country_code = 805')

df4 = df3.dropDuplicates(['duns', 'append_year', 'append_month'])

df4[['duns', 'append_year', 'append_month', 'export']].write.mode('overwrite').saveAsTable('workarea.us_export_propensity_exporters')